<a href="https://colab.research.google.com/github/Rameshkumar-V/handson-smolagents/blob/main/database_agent_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

step 1 : Installing Packages

In [ ]:
!pip install smolagents python-dotenv sqlalchemy --upgrade -q

In [8]:
import os
os.environ["OPENAI_API_KEY"]=""

step 2 : Loading Environment

In [3]:
from dotenv import load_dotenv
load_dotenv()

False

In [4]:
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    Float,
    insert,
    inspect,
    text,
)

engine = create_engine("sqlite:///:memory:")
metadata_obj = MetaData()

def insert_rows_into_table(rows, table, engine=engine):
    for row in rows:
        stmt = insert(table).values(**row)
        with engine.begin() as connection:
            connection.execute(stmt)

table_name = "receipts"
receipts = Table(
    table_name,
    metadata_obj,
    Column("receipt_id", Integer, primary_key=True),
    Column("customer_name", String(16), primary_key=True),
    Column("price", Float),
    Column("tip", Float),
)
metadata_obj.create_all(engine)

rows = [
    {"receipt_id": 1, "customer_name": "Alan Payne", "price": 12.06, "tip": 1.20},
    {"receipt_id": 2, "customer_name": "Alex Mason", "price": 23.86, "tip": 0.24},
    {"receipt_id": 3, "customer_name": "Woodrow Wilson", "price": 53.43, "tip": 5.43},
    {"receipt_id": 4, "customer_name": "Margaret James", "price": 21.11, "tip": 1.00},
]
insert_rows_into_table(rows, receipts)

step 4: Get Table Information for LLM

In [5]:
inspector = inspect(engine)
columns_info = [(col["name"], col["type"]) for col in inspector.get_columns("receipts")]

table_description = "Columns:\n" + "\n".join([f"  - {name}: {col_type}" for name, col_type in columns_info])
print(table_description)

Columns:
  - receipt_id: INTEGER
  - customer_name: VARCHAR(16)
  - price: FLOAT
  - tip: FLOAT


In [6]:
from smolagents import tool

@tool
def sql_engine(query: str) -> str:
    """
    Allows you to perform SQL queries on the table. Returns a string representation of the result.
    The table is named 'receipts'. Its description is as follows:
        Columns:
        - receipt_id: INTEGER
        - customer_name: VARCHAR(16)
        - price: FLOAT
        - tip: FLOAT

    Args:
        query: The query to perform. This should be correct SQL.
    """
    output = ""
    with engine.connect() as con:
        rows = con.execute(text(query))
        for row in rows:
            output += "\n" + str(row)
    return output

In [9]:
import os
from smolagents import CodeAgent
from smolagents.models import OpenAIServerModel

# Set OpenRouter API key
api_key = os.getenv("OPENAI_API_KEY")  # Ensure your OpenRouter API key is set

# Initialize SmolAgent model
model = OpenAIServerModel(
    model_id="qwen/qwen2.5-vl-72b-instruct:free",
    api_key=api_key,
    api_base="https://openrouter.ai/api/v1"
)

# Initialize the agent with tools
agent = CodeAgent(
    tools=[sql_engine],  # Ensure sql_engine is defined
    model=model
)

# Run the agent with a query
response = agent.run("Can you give me the name of the client who got the most expensive receipt?")

print(response)


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Can you give me the name of the client who got the most expensive receipt?                                      │
│                                                                                                                 │
╰─ OpenAIServerModel - qwen/qwen2.5-vl-72b-instruct:free ─────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  query = "SELECT customer_name FROM receipts ORDER BY price DESC LIMIT 1"                                         
  result = sql_engine(query=query)                                                                                 
  print(result)                                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:

('Woodrow Wilson',)

Out: None

[Step 0: Duration 7.01 seconds| Input tokens: 2,131 | Output tokens: 92]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Woodrow Wilson")                                                                                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Woodrow Wilson

[Step 1: Duration 3.96 seconds| Input tokens: 4,448 | Output tokens: 155]

Woodrow Wilson


In [11]:
# Run the agent with a query
response = agent.run("Can you give me First Person Name ")

print(response)


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Can you give me First Person Name                                                                               │
│                                                                                                                 │
╰─ OpenAIServerModel - qwen/qwen2.5-vl-72b-instruct:free ─────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  query = "SELECT customer_name FROM receipts LIMIT 1"                                                             
  result = sql_engine(query=query)                                                                                 
  print(result)                                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:

('Alan Payne',)

Out: None

[Step 0: Duration 7.96 seconds| Input tokens: 2,123 | Output tokens: 114]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Alan")                                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Alan

[Step 1: Duration 4.09 seconds| Input tokens: 4,449 | Output tokens: 175]

Alan


In [12]:
# Run the agent with a query
response = agent.run("Can you give me who got most tip theirs name ")

print(response)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Can you give me who got most tip theirs name                                                                    │
│                                                                                                                 │
╰─ OpenAIServerModel - qwen/qwen2.5-vl-72b-instruct:free ─────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  query = "SELECT customer_name FROM receipts ORDER BY tip DESC LIMIT 1"                                           
  result = sql_engine(query=query)                                                                                 
  print(result)                                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:

('Woodrow Wilson',)

Out: None

[Step 0: Duration 5.90 seconds| Input tokens: 2,126 | Output tokens: 77]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Woodrow Wilson")                                                                                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Woodrow Wilson

[Step 1: Duration 24.10 seconds| Input tokens: 4,423 | Output tokens: 118]

Woodrow Wilson


In [13]:
# Run the agent with a query
response = agent.run("Can you give me higher character container person name ")

print(response)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Can you give me higher character container person name                                                          │
│                                                                                                                 │
╰─ OpenAIServerModel - qwen/qwen2.5-vl-72b-instruct:free ─────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  print("Could you please provide more details or context? For example, do you want the name with the highest      
  number of characters from a specific list or database?")                                                         
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Could you please provide more details or context? For example, do you want the name with the highest number of 
characters from a specific list or database?

Out: None

[Step 0: Duration 7.27 seconds| Input tokens: 2,125 | Output tokens: 89]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Woodrow Wilson")                                                                                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Woodrow Wilson

[Step 2: Duration 8.92 seconds| Input tokens: 7,004 | Output tokens: 244]

Woodrow Wilson
